# 🎵 Classification des Genres Musicaux
## Notebook 1: Exploration des Données

**Objectif:** Explorer le dataset GTZAN et comprendre les caractéristiques des fichiers audio.

---

## 1. Configuration et Imports

In [ ]:
# Imports standards
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
import IPython.display as ipd
from pathlib import Path

# Ajouter le répertoire src au path
sys.path.insert(0, '..')

# Imports du projet
from src.config import Config
from src.data_loader import DataLoader
from src.visualization import Visualizer

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
%matplotlib inline

print("✅ Imports réussis!")

In [ ]:
# Afficher la configuration
Config.print_config()

## 2. Chargement et Exploration du Dataset

In [ ]:
# Initialiser le DataLoader
loader = DataLoader()

# Scanner le dataset
df = loader.scan_dataset()
df.head(10)

In [ ]:
# Afficher le résumé
loader.print_dataset_summary(df)

In [ ]:
# Distribution des genres
visualizer = Visualizer()
visualizer.plot_genre_distribution(df)
plt.show()

## 3. Exploration d'un Fichier Audio

In [ ]:
# Sélectionner un fichier par genre
sample_files = df.groupby('genre').first().reset_index()
sample_files[['genre', 'filename']]

In [ ]:
# Charger un exemple de chaque genre
def explore_audio_file(filepath, genre):
    """Explore un fichier audio."""
    # Charger l'audio
    y, sr = librosa.load(filepath, sr=Config.SAMPLE_RATE, duration=30)
    
    print(f"\n🎵 Genre: {genre.upper()}")
    print(f"   Fichier: {Path(filepath).name}")
    print(f"   Durée: {len(y)/sr:.2f} secondes")
    print(f"   Échantillons: {len(y)}")
    print(f"   Fréquence: {sr} Hz")
    
    # Écouter l'audio (5 premières secondes)
    display(ipd.Audio(y[:sr*5], rate=sr))
    
    return y, sr

In [ ]:
# Explorer un fichier de chaque genre
audio_samples = {}

for idx, row in sample_files.iterrows():
    y, sr = explore_audio_file(row['filepath'], row['genre'])
    audio_samples[row['genre']] = (y, sr)

## 4. Visualisation des Signaux Audio

In [ ]:
# Choisir un genre pour l'analyse détaillée
genre_to_analyze = 'jazz'  # Changez cette valeur

if genre_to_analyze in audio_samples:
    y, sr = audio_samples[genre_to_analyze]
    
    # Visualisation complète
    visualizer.plot_audio_analysis(y, sr, title=f"Analyse Audio - {genre_to_analyze.upper()}")
    plt.show()
else:
    print(f"Genre '{genre_to_analyze}' non disponible.")

In [ ]:
# Comparer les formes d'onde de différents genres
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

genres_to_compare = ['classical', 'metal', 'hiphop', 'jazz', 'pop', 'rock']

for ax, genre in zip(axes, genres_to_compare):
    if genre in audio_samples:
        y, sr = audio_samples[genre]
        librosa.display.waveshow(y[:sr*5], sr=sr, ax=ax, color='steelblue')
        ax.set_title(genre.upper())
        ax.set_xlabel('')

plt.suptitle("Comparaison des Formes d'Onde (5 premières secondes)", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Comparaison des Spectrogrammes

In [ ]:
# Comparer les mel-spectrogrammes de différents genres
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for ax, genre in zip(axes, genres_to_compare):
    if genre in audio_samples:
        y, sr = audio_samples[genre]
        
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        
        img = librosa.display.specshow(mel_spec_db, sr=sr, x_axis='time', y_axis='mel', ax=ax)
        ax.set_title(genre.upper())

plt.suptitle("Comparaison des Mel-Spectrogrammes", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Statistiques de Base par Genre

In [ ]:
# Calculer quelques statistiques de base
stats = []

for genre, (y, sr) in audio_samples.items():
    # Tempo
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    tempo_val = float(tempo[0]) if isinstance(tempo, np.ndarray) else float(tempo)
    
    # Zero crossing rate
    zcr = librosa.feature.zero_crossing_rate(y)[0]
    
    # RMS Energy
    rms = librosa.feature.rms(y=y)[0]
    
    # Spectral centroid
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    
    stats.append({
        'Genre': genre,
        'Tempo (BPM)': tempo_val,
        'ZCR Mean': np.mean(zcr),
        'RMS Mean': np.mean(rms),
        'Centroid Mean': np.mean(centroid)
    })

stats_df = pd.DataFrame(stats)
stats_df = stats_df.round(4)
stats_df

In [ ]:
# Visualiser les statistiques
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

metrics = ['Tempo (BPM)', 'ZCR Mean', 'RMS Mean', 'Centroid Mean']
colors = sns.color_palette('husl', len(stats_df))

for ax, metric in zip(axes.flatten(), metrics):
    bars = ax.bar(stats_df['Genre'], stats_df[metric], color=colors)
    ax.set_title(metric)
    ax.set_xlabel('')
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.suptitle("Statistiques Audio par Genre", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Conclusions de l'Exploration

### Observations:

1. **Distribution des genres:** Le dataset est équilibré avec 100 fichiers par genre.

2. **Tempo:** Les genres comme le metal et le disco ont tendance à avoir des tempos plus élevés.

3. **Spectrogrammes:** On observe des différences visuelles notables entre genres:
   - Classique: fréquences plus basses, harmoniques claires
   - Metal: énergie élevée sur tout le spectre
   - Hip-hop: basses prononcées

4. **Zero Crossing Rate:** Le rock et le metal montrent des ZCR plus élevés (plus de "bruit").

### Prochaines étapes:
- Extraire les caractéristiques audio (MFCC, chroma, etc.)
- Entraîner les modèles de classification

In [ ]:
print("✅ Exploration terminée!")
print("\n📌 Passez au notebook 02_feature_extraction.ipynb pour l'extraction des caractéristiques.")